<a href="https://colab.research.google.com/github/QaziSaim/AI-Projects/blob/main/Concurrent_Query_Resolution_System_Using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%%capture
!pip install crewai
!pip install crewai-tools
!pip install langchain_google_genai


In [37]:
import os
import asyncio

In [38]:
from google.colab import userdata
gemini_api=userdata.get('GEMINI_API')
groq_api = userdata.get('GROQ_API')

In [39]:
from crewai import Agent, Crew, Task, LLM, Process

In [60]:
llm1 = LLM(
    model='gemini/gemini-2.5-flash',
    temperature=0.7,
    api_key=gemini_api
)
llm2 = LLM(
    model='groq/Gemma2-9b-It',
    temperature=0.5,
    api_key = groq_api
)

In [61]:
query_resolution = Agent(
    llm=llm1,
    role = 'Query Resolver',
    backstory="An AI agent that resolves customer queries efficiently and professionally.",
    goal="Resolve customer queries accurately and provide helpful solutions.",
    verbose=True

)

In [62]:
summary_agent = Agent(
    llm=llm2,
    role="Summary Generator",
    backstory="An AI agent that summarizes the resolution of customer queries.",
    goal="Provide a concise summary of the query resolution process.",
    verbose=True
)


In [63]:
resolution_task = Task(
    description="Resolve the customer query: {query}.",
    expected_output="A detailed resolution for the customer query.",
    agent=query_resolution
)

summary_task = Task(
    description="Summarize the resolution of the customer query: {query}.",
    expected_output="A concise summary of the query resolution.",
    agent=summary_agent
)


In [64]:
async def execute_query(query: str):
    crew = Crew(
        agents=[query_resolution, summary_agent],
        tasks=[resolution_task, summary_task],
        process=Process.sequential,
        verbose=True
    )
    result = await crew.kickoff_async(inputs={"query": query})
    return result


In [65]:
async def handle_two_queries(query_1: str, query_2: str):
    # Run both queries concurrently
    results = await asyncio.gather(
        execute_query(query_1),
        execute_query(query_2)
    )
    return results


In [66]:
query_1 = "I am unable to log in to my account. It says 'Invalid credentials', but I am sure I am using the correct username and password."
query_2 = "The payment gateway is not working. Also, a weird error message is displayed. My card has been charged, but the transaction is not going through."


In [67]:
try:
    loop = asyncio.get_event_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)


In [68]:
# Check if the event loop is already running
if loop.is_running():
    # If the loop is running, use `nest_asyncio` to allow re-entrant event loops
    import nest_asyncio
    nest_asyncio.apply()


In [69]:
# Run the async function
results = loop.run_until_complete(handle_two_queries(query_1, query_2))

# Print the results
for i, result in enumerate(results):
    print(f"Result for Query {i+1}:")
    print(result)
    print("\n---\n")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1abf4367-27f0-4f01-bba2-13cd49ebd1f9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 603acf26-52ed-4309-b8ea-66ade303b778                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Query Resolver                                                                                          │
│                                                                                                                 │
│  Task: Resolve the customer query: The payment gateway is not working. Also, a weird error message is           │
│  displayed. My card has been charged, but the transaction is not going through..                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Query Resolver                                                                                          │
│                                                                                                                 │
│  Task: Resolve the customer query: I am unable to log in to my account. It says 'Invalid credentials', but I    │
│  am sure I am using the correct username and password..                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summary Generator                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The customer was experiencing an issue logging into their account due to an 'Invalid credentials' message.     │
│  They were advised to double-check their username and password for typos, case sensitivity, and to ensure they  │
│  were using the correct login method (username or email).  They were also instructed to clear their browser     │
│  cache and cookies, try a different browser or incognito mode, and reset their password if the issue            │
│  persisted.  The customer was provided with detailed instructions on how to reset their password and was asked  │
│  to provide additional information if they continued to have problems.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 02e23d23-0284-4866-a1ed-91f1b8719715                                                                     │
│  Agent: Summary Generator                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1abf4367-27f0-4f01-bba2-13cd49ebd1f9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The customer was experiencing an issue logging into their account due to an 'Invalid             │
│  credentials' message.  They were advised to double-check their username and password for typos, case           │
│  sensitivity, and to ensure they were using the correct login method (username or email).  They were also       │
│  instructed to clear their browser cache and cookies, try a different browser or incognito mode, and reset      │
│  their password if the issue persisted.  The customer was provided with detailed instructions on how to reset   │
│  their password and was asked to provide additional information if they continued to have problems.             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Result for Query 1:
The customer was experiencing an issue logging into their account due to an 'Invalid credentials' message.  They were advised to double-check their username and password for typos, case sensitivity, and to ensure they were using the correct login method (username or email).  They were also instructed to clear their browser cache and cookies, try a different browser or incognito mode, and reset their password if the issue persisted.  The customer was provided with detailed instructions on how to reset their password and was asked to provide additional information if they continued to have problems.

---

Result for Query 2:
The customer was experiencing a "weird error message" and their card was charged despite the transaction failing. The support team explained that this is often due to a temporary hold placed by the customer's bank. They advised the customer to check their bank statement for a pending charge, avoid re-attempting the transaction immediately, and pro